<a href="https://colab.research.google.com/github/nguyen-nhat-mai/methane-leak-detection/blob/main/Web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build web app for classifier model

In [1]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


## Ver 1: Upload and infer image by image



Reference:

https://towardsdatascience.com/create-an-image-classification-web-app-using-pytorch-and-streamlit-f043ddf00c24#2b4c
https://www.youtube.com/watch?v=NEhrkeF2o_M

In [ ]:
# Write the web app
%%writefile app.py
import streamlit as st
from PIL import Image
from torchvision import models, transforms
import torch

#---------------------- TO UPDATE WITH FINAL MODEL-----------------------#
def predict(image_path):
    # Load model
    best_model = models.resnet101(pretrained=True)
    # Define transformation
    transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
      )])
    # Load data
    img = Image.open(image_path)
    batch_t = torch.unsqueeze(transform(img), 0)
    # Do inference
    best_model.eval()
    out = best_model(batch_t)
    # Load all of the classes => Yes/ No for methane case
    with open('imagenet_classes.txt') as f:
        classes = [line.strip() for line in f.readlines()]
    # Calculate the probability
    prob = torch.nn.functional.softmax(out, dim=1)[0] * 100
    _, indices = torch.sort(out, descending=True)
    # Return the classes and corresponding probability
    return [(classes[idx], prob[idx].item()) for idx in indices[0][:2]]

# ------------------------------WEB APP----------------------------------#
st.set_option('deprecation.showfileUploaderEncoding', False)

st.title("Localize Methane Leaks")
st.write("")

file_up = st.file_uploader("Upload an image", type="jpg")

if file_up is not None:
    image = Image.open(file_up)
    st.image(image, caption='Uploaded Image.', use_column_width=True)
    st.write("")
    st.write("Just a second...")
    labels = predict(file_up)

    # print out the prediction labels with scores
    for i in labels:
        st.write("Prediction:", i[0], ",   Probability (%): ", i[1])

Overwriting app.py


In [ ]:
# Run app.py and made available on a local URL
!streamlit run app.py & npx localtunnel --port 8501

[#######...........] / extract:localtunnel: verb lock using /root/.npm/_locks/s


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.143.211:8501

npx: installed 22 in 4.458s
your url is: https://chatty-tigers-relate.loca.lt
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Ver 2: Upload batch of images and export results

In [ ]:
# Write the web app
%%writefile app.py
import streamlit as st
from PIL import Image
from torchvision import models, transforms
import torch

#---------------------- TO UPDATE WITH FINAL MODEL-----------------------#
def predict(image_path):
    # Load model
    best_model = models.resnet101(pretrained=True)
    # Define transformation
    transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225]
      )])
    # Load data
    img = Image.open(image_path)
    batch_t = torch.unsqueeze(transform(img), 0)
    # Do inference
    best_model.eval()
    out = best_model(batch_t)
    # Load all of the classes => Yes/ No for methane case
    with open('imagenet_classes.txt') as f:
        classes = [line.strip() for line in f.readlines()]
    # Calculate the probability
    prob = torch.nn.functional.softmax(out, dim=1)[0] * 100
    _, indices = torch.sort(out, descending=True)
    # Return the classes and corresponding probability
    return [(classes[idx], prob[idx].item()) for idx in indices[0][:2]]

# ------------------------------WEB APP----------------------------------#
st.set_option('deprecation.showfileUploaderEncoding', False)

st.title("Localize Methane Leaks")
st.write("")

file_up = st.file_uploader("Upload your images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)

if file_up is not None:
    for img_file in file_up:
        image = Image.open(img_file)
        st.image(image, caption='Uploaded Image.', use_column_width=True)
        st.write("")
        st.write("Just a second...")
        labels = predict(img_file)

        # print out the prediction labels with scores
        for i in labels:
            st.write("Prediction:", i[0], ",   Probability (%): ", i[1])

In [ ]:
# Run app.py and made available on a local URL
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.73.128.93:8501

npx: installed 22 in 5.297s
your url is: https://flat-experts-shine.loca.lt
